In [1]:
#!/usr/bin/env python
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, ADASYN
from skimage import io
from sklearn.feature_selection import VarianceThreshold

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
feature_name_file = 'Feature_name_'+case_name+'.csv'
table = pd.read_csv( case_path +case_name+'.csv')
preprocess_file = preprocessed_path
preprocessed_file_path = preprocess_file+ '/' +case_name

if not os.path.exists(preprocessed_file_path):
    os.makedirs(preprocessed_file_path)

annote = '/home/DL-based-Tumor-Classification/04072018_annotation.csv'

folds = 10
im_size = 98

In [3]:
def embedding_2d(features_train, features_test, fold_idx, preprocessed_file_path, labels_train, labels_test):
    print('embedding to 2D image, fold:', fold_idx)
    features_padded = np.zeros(im_size*im_size)
    subfolder_path = os.path.join(preprocessed_file_path + '/img_fold' + str(fold_idx))
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
    subfolder_train_path = subfolder_path + '/train'
    if not os.path.exists(subfolder_train_path):
        os.makedirs(subfolder_train_path)
    subfolder_test_path = subfolder_path + '/test'
    if not os.path.exists(subfolder_test_path):
        os.makedirs(subfolder_test_path)
    pd.DataFrame(labels_train).to_csv(os.path.join(subfolder_train_path, 'labels_train.csv'))
    pd.DataFrame(labels_test).to_csv(os.path.join(subfolder_test_path, 'labels_test.csv'))
    for i in range(features_train.shape[0]):
        features_padded[range(features_train.shape[1])] = (features_train[i, :]- min(features_train[i,:]))/max(features_train[i, :]) *255
        features_train_tmp = features_padded.reshape(im_size, im_size)
        file_path = subfolder_train_path + '/'+str(i) + '.png'
        io.imsave(file_path, np.uint8(features_train_tmp))
    for j in range(features_test.shape[0]):
        features_padded[range(features_test.shape[1])] = (features_test[j, :]- min(features_train[i,:]))/max(features_test[j, :]) *255
        features_test_tmp = features_padded.reshape(im_size, im_size)
        file_path = subfolder_test_path + '/' + str(j) + '.png'
        io.imsave(file_path, np.uint8(features_test_tmp))

In [4]:
def embedding_1d(features_train, features_test, fold_idx, preprocessed_file_path, labels_train, labels_test):
    print('embedding to 1D image, fold:', fold_idx)
    features_padded = np.zeros(im_size*im_size)
    subfolder_path = os.path.join(preprocessed_file_path + '/img_fold' + str(fold_idx))
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
    subfolder_train_path = subfolder_path + '/train'
    if not os.path.exists(subfolder_train_path):
        os.makedirs(subfolder_train_path)
    subfolder_test_path = subfolder_path + '/test'
    if not os.path.exists(subfolder_test_path):
        os.makedirs(subfolder_test_path)
    pd.DataFrame(labels_train).to_csv(os.path.join(subfolder_train_path, 'labels_train.csv'))
    pd.DataFrame(labels_test).to_csv(os.path.join(subfolder_test_path, 'labels_test.csv'))
    for i in range(features_train.shape[0]):
        features_padded[range(features_train.shape[1])] = (features_train[i, :] - min(features_train[i,:]))/max(features_train[i, :]) *255
        features_train_tmp = features_padded.reshape(1, im_size*im_size)
        file_path = subfolder_train_path + '/'+str(i) + '.png'
        io.imsave(file_path, np.uint8(features_train_tmp))
    for j in range(features_test.shape[0]):
        features_padded[range(features_test.shape[1])] = (features_test[j, :]- min(features_train[i,:]))/max(features_test[j, :]) *255
        features_test_tmp = features_padded.reshape(1, im_size*im_size)
        file_path = subfolder_test_path + '/' + str(j) + '.png'
        io.imsave(file_path, np.uint8(features_test_tmp))

In [5]:
def feature_selection(big_table,preprocessed_file_path):
    feature_name = list(big_table)
    feature_name_o = list(big_table.iloc[0,1:])
    labels = np.array(big_table.iloc[1:, 0])
#     annotation = pd.read_csv.loc[:, "GeneID"])
#     gene_id_chr_annotation = list(av(annotation_path, dtype=str)
#     gene_id_annotation = list(annotationnotation.loc[:, "chromosome"])
    gene_id_original = []
    idx1 = []
    idx1_annotation = []
    k = 0
    for name in feature_name_o:
#         symbol, gene_id = name.split("|", 1)
        gene_id_original.append(name)
#         if gene_id in gene_id_annotation:
        idx1.append(k)
#             idx1_annotation.append(gene_id_annotation.index(gene_id))
#         print('compare with annotation, progress :', k / len(feature_name))
        k = k + 1
    features_raw = np.array(big_table.iloc[1:, 1:], dtype=float)
    print("feature_raw shape : ", features_raw.shape)
    features = np.log2(1.0 + features_raw)
    features[np.where(features <= 1)] = 0
    # values corresponding to existing genes (1st filtering)
    features_filtered = np.array(list(features[:, idx1_tmp] for idx1_tmp in idx1)).transpose()
    print("features_filtered shape : ", features_filtered.shape)
    feature_name_filtered = list(feature_name[i] for i in idx1)
#     gene_id_chr = list(gene_id_chr_annotation[i] for i in idx1_annotation)
    # sort the features based on the chr number
#     idx_sorted = sort_feature(gene_id_chr)
    feature_name_sorted = list(feature_name_filtered[j] for j in idx1)
    features_sorted = np.array(list(features_filtered[:, i] for i in idx1)).transpose()
    print("features_sorted shape : " , features_sorted.shape)
#     print('features have been sorted based on chromosome')
    selector = VarianceThreshold(threshold=1)
    selector.fit(features_filtered)
    idx2 = selector.get_support()
    idx2_num = selector.get_support(indices=True)
    # numpy is different from list
    features = features_sorted[:, idx2]
    print("final features shape: ", features.shape)
    feature_name_final = list(feature_name_sorted[i] for i in idx2_num)
    feature_name_path = os.path.join(preprocessed_file_path, feature_name_file)
    pd.DataFrame(feature_name_final).to_csv(feature_name_path)
    print('features are selected, the selected gene name are saved at', feature_name_path)
    return features, labels
   
def sort_feature(chr_filtered):
    idx_list = list(range(len(chr_filtered)))
    big_list_chr = zip(idx_list, chr_filtered)  # combine a list of indices and a list of chromosomes
    tmp = sorted(big_list_chr, key=lambda x: x[1])  # sort based on the chromosome
    idx, _ = zip(*tmp)
    return idx


def kfold_split(big_table, folds, preprocessed_file_path):
    features, labels = feature_selection(big_table, preprocessed_file_path)
    print('features have been read')
    skf = StratifiedKFold(n_splits=folds, shuffle=True)
    index_list = list(skf.split(features, labels))
    return features, labels, index_list

def over_sampling(training_data, training_label):
    """
    This function is used only for classification
    :param training_data:
    :param training_label:
    :return: The training data and training label after oversampling using SMOTE algorithm
    """
    training_data_resampled, training_label_resampled = SMOTE(ratio='minority', random_state=42, kind='svm', n_jobs=12)\
        .fit_sample(training_data, training_label)
    return training_data_resampled, training_label_resampled

feature_raw shape :  (182, 20531)
features_filtered shape :  (182, 20531)
features_sorted shape :  (182, 20531)
final features shape:  (182, 8969)
features are selected, the selected gene name are saved at /home/DL-based-Tumor-Classification/Datasets/Newest_case/2D_reform/Feature_name_Carcinoma+Adenocarcinoma.csv
features have been read


embedding to 2D image, fold: 0
embedding to 2D image, fold: 1
embedding to 2D image, fold: 2
embedding to 2D image, fold: 3
embedding to 2D image, fold: 4
embedding to 2D image, fold: 5
embedding to 2D image, fold: 6
embedding to 2D image, fold: 7
embedding to 2D image, fold: 8
embedding to 2D image, fold: 9
done
